<a href="https://colab.research.google.com/github/judeha/addnoise/blob/main/Apply_Noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Import datasets from hugging face
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
conll=load_dataset("conll2003")

In [37]:
# from copy import deepcopy
# conll=deepcopy(conll)

In [38]:
# Import dependencies
import numpy as np
import pandas as pd
import random
import math
import json
import os

from bisect import bisect_left

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

path = ('/content/drive/My Drive/Dell Lab/')

In [40]:
# Load substitution probabilities
subst_prob = pd.read_csv(path + 'subst_prob.csv')

with open(path + 'subst_cdf.json') as f:
  subst_cdf = json.load(f)

In [41]:
# Load insertion probabilities

with open(path + 'alph.json') as f:
  alph = json.load(f)

with open(path + 'ALPH.json') as f:
  ALPH = json.load(f)

non_alph = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
           '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
           ':', ';', '<', '=', '>', '?', '@',
           '[', '\\', ']',
           '{', '|', '}', '~']

In [ ]:
print(subst_prob)
print(subst_cdf)

In [ ]:
char_list = list(subst_prob.columns.values)
char_list.pop(0)
print(char_list)

In [65]:
# NEXT: is this a bad or good article?
random.seed(42)

# Substitution function
def substitute(c: str):
    if c in char_list:
      # identifies insertion index to maintain order
      i = bisect_left(subst_cdf[c], random.random())
      return subst_cdf[c][i+1]
    else: return c
  
# Insertion function 
def insert():
  if random.random < 0.55:
    i = bisect_left(alph, random.random())
    if random.random < 0.75:
      return alph[i+1]
    else: return ALPH[i+1]
  else:
    return random.choice(non_alph)

# Simulate deletion of spaces between words by combining
def combine(text, index):
  if index > 0:
    word, label = text[index]
    prev_word, prev_label = text[index-1]
    new_word = prev_word + word
    new_label = prev_label
    return new_word, new_label
  else: return text[index]

In [66]:
def add_noise_to_word(word, label):
  chars = list(word)
  for i, count in enumerate(word):
    if random.random() < 0.02:
      chars[i] = ''
      continue
    if random.random() < subst_prob[i]:
      chars[i] = substitute(i)
    if random.random() < 0.02:
      chars[i] = chars[i] + insert()

    # insert space
    if random.random() < 0.02:
      chars[i] = chars[i] + ' '
  return ''.join(chars), label

In [67]:
def add_noise_to_sentence(sentence: list):
  for i in len(sentence):
    if random.random < 0.02:
      sentence[i-1] = combine(sentence, i)
      del sentence[i]
    # Add noise to each word
    if sentence[i]:
      sentence[i] = add_noise_to_word(sentence[i])
    return sentence

def add_noise(data):
  return (add_noise_to_sentence(sentence) for sentence in data)

In [ ]:
# Combine tokens into text
# from nltk.tokenize.treebank import TreebankWordDetokenizer
# corpus = conll['train']['tokens']

# for i in range(len(corpus)):
#   corpus[i] = TreebankWordDetokenizer().detokenize(corpus[i])

# print(corpus)

In [ ]:
# Track words and labels
conll_train, conll_val, conll_test = (
    [
        list(zip(sentence_words, sentence_labels))
        for sentence_words, sentence_labels
        in zip(conll[split]['tokens'], conll[split]['ner_tags'])
    ]
    for split in ('train', 'validation', 'test')
)

print(conll_train)

In [69]:
conll_train_noisy = add_noise(conll_train)
conll_val_noisy = add_noise(conll_val)
conll_test_noisy = add_noise(conll_test)

In [70]:
id2label = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
    7: 'B-MISC',
    8: 'I-MISC'
}

In [ ]:
# Save
with open('noisy_conll_train.txt', 'w') as f:
  for sentence in conll_train_noisy:
    for word, label in sentence:
      f.write(f'{word} {id2label[label]}\n')
    f.write('\n')

In [ ]:

with open('noisy_conll-2003_val.txt', 'w') as f:
    for sentence in conll_val_noisy:
        for word, label in sentence:
            f.write(f'{word} {id2label[label]}\n')
        f.write('\n')
with open('noisy_conll-2003_test.txt', 'w') as f:
    for sentence in conll_test_noisy:
        for word, label in sentence:
            f.write(f'{word} {id2label[label]}\n')
        f.write('\n')